# Using **mem0** with Azure resources and API keys

This example demonstrates how to utilize **mem0** with API keys

## Prerequisites

### 1. Install Mem0

To use Mem0, install it using pip:

```bash
pip install "mem0ai"
```

### 2. Obtain API keys

### *Azure OpenAI setup*
- In the Azure portal, go to your **Azure OpenAI** resource.
- In the left hand channel expand *Resource Management*.
- Select *Keys and Endpoint*
- Copy either Key 1 or Key 2 to your clipboard and paste in the `your-azure-openai-api-key` constant below.
  - If local authentication is disabled for your service you must use an Azure Identity credential, see the help docs and the Azure Identity example for more information.

### *Azure Search setup*
- In the Azure portal, go to your **Azure AI Search** resource.
- In the left hand channel expand *Settings*.
- Select *Keys*
- Copy either Key 1 or Key 2 to your clipboard and paste in the `your-azure-search-api-key` constant below.
  - If *Role-based access control* is selected, no API keys will be available and you must use an Azure Identity credential, see the help docs and the Azure Identity example for more information. 

## Imports and Constants

In [ ]:
import json
from mem0 import Memory

# Change these constants to your Azure OpenAI and Azure AI Search configurations
azure_openai_api_key = "<your-azure-openai-api-key>"

azure_openai_chat_model = "<your-azure-model>"  # e.g., "gpt-4o
azure_openai_endpoint = "<your-azure-endpoint>"  # e.g., "https://your-azure-openai-endpoint.openai.azure.com/"
azure_openai_chat_deployment = "<your-azure-deployment>"  # e.g., "gpt-4o"
azure_openai_chat_api_version = "<your-api-version>"  # e.g., "2024-12-01-preview"

azure_openai_embedding_model = "<your-embedding-model-name>"  # e.g., "text-embedding-3-large"
azure_openai_embedding_endpoint = (
    "<your-azure-endpoint>"  # e.g., "https://your-azure-openai-endpoint.openai.azure.com/"
)
azure_openai_embedding_deployment = "<your-embedding-deployment>"  # e.g., "text-embedding-3-large"
azure_openai_embedding_api_version = "<your-embedding-api-version>"  # e.g., "2024-02-01"

azure_search_api_key = "<your-azure-search-api-key>"

azure_search_service_name = "<your-azure-search-service-name>"  # e.g., "my-aiss" (name of the Azure AI Search resource)

# Create **mem0** configuration

1. LLM uses Azure OpenAI
2. Embedder uses Azure OpenAI
3. Vector Store uses Azure AI Search

In [ ]:
config = {
    "llm": {
        "provider": "azure_openai",
        "config": {
            "model": azure_openai_chat_model,
            "temperature": 0.1,
            "max_tokens": 2000,
            "azure_kwargs": {
                "azure_endpoint": azure_openai_endpoint,
                "azure_deployment": azure_openai_chat_deployment,
                "api_version": azure_openai_chat_api_version,
                "api_key": azure_openai_api_key,
            },
        },
    },
    "embedder": {
        "provider": "azure_openai",
        "config": {
            "model": azure_openai_embedding_model,
            "azure_kwargs": {
                "azure_endpoint": azure_openai_embedding_endpoint,
                "azure_deployment": azure_openai_embedding_deployment,
                "api_version": azure_openai_embedding_api_version,
                "api_key": azure_openai_api_key,
            },
        },
    },
    "vector_store": {
        "provider": "azure_ai_search",
        "config": {
            "service_name": azure_search_service_name,
            "collection_name": "mem0",
            "embedding_model_dims": 3072,
            "api_key": azure_search_api_key,
        },
    },
}

print("Configuration for Azure OpenAI and Azure AI Search.")
print(json.dumps(config, indent=2))

## Memory initialization

In [ ]:
m = Memory.from_config(config_dict=config)

## Store memories

In [ ]:
# create a memory as a set of chat messages
messages = [
    {
        "role": "user",
        "content": "I'm planning to watch a movie tonight. Any recommendations?",
    },
    {
        "role": "assistant",
        "content": "How about a thriller movies? They can be quite engaging.",
    },
    {
        "role": "user",
        "content": "I'm not a big fan of thriller movies but I love sci-fi movies.",
    },
    {
        "role": "assistant",
        "content": "Got it! I'll avoid thriller recommendations and suggest sci-fi movies in the future.",
    },
]
# Store inferred memories (default behavior)
result = m.add(messages, user_id="alice", metadata={"category": "movie_recommendations"})

## Search memories

In [ ]:
# search stored memories
for result in m.search("what does alice love?", user_id="alice")["results"]:
    print(result["memory"], result["score"])